In [1]:
import sys
import pandas as pd
sys.path.append('/Users/smallhamster/Documents/research/jokesclustering/') # ваш путь до корня проекта
from components import FasttextVectorizer, KmeansSimpleClusterExtractor
from vector_clustering.abstract.model import Model
from vector_clustering.data.manager import get_jokes_as_dataframe, load_pandas_csv
from sklearn.cluster import KMeans

### 1. Инициализируем все необходимое

In [2]:
jokes = get_jokes_as_dataframe()
jokes.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Только у нас фраза ""поправить здоровье"" означа..."
4,"Одна белка случайно попробовала пиво и поняла,..."


In [3]:
selected = load_pandas_csv('100_manual_selection.csv')
selected.head()

,joke_index,joke_text,joke_topics
0,75622,Вот и минул ещё один день дурака. Но хватит о ...,политика
1,88009,Китайские космонавты после успешно завершенной...,национальность
2,20296,"Украина - единственная страна мира, где грипп ...",политика
3,45900,Временно исполняющий обязанности президента ма...,политика
4,52912,"Решение наболевшего дела проще, чем вы там себ...",жизнь


In [4]:
topics = selected.joke_topics.values
unique_topics = set()
for topic in topics:
    unique_topics.update([x.strip() for x in topic.split(',')])

Ручной анализ тем в 100 случайных шутках, показал, что мы наблюдаем следующие темы:

In [5]:
unique_topics

{'алкоголь',
 'армия',
 'деньги',
 'еда',
 'жадность',
 'животные',
 'жизнь',
 'зависть',
 'игра слов',
 'имя',
 'история',
 'компьютер',
 'лгбт',
 'лень',
 'мат',
 'медицина',
 'менты',
 'мигранты',
 'наркота',
 'национальность',
 'непонятно',
 'образование',
 'отдых',
 'отношения',
 'плагиат',
 'погода',
 'пол',
 'политика',
 'пошлятина',
 'праздники',
 'преступление',
 'работа',
 'расизм',
 'сарказм',
 'семья',
 'спорт',
 'студенты',
 'туристы',
 'тюрьма',
 'штирлиц',
 'экономика',
 'эстрада'}

In [6]:
len(unique_topics)

42

предположим, что на самом деле тем на 20% больше, откуда следует, что будем искать следующее число кластеров:

In [8]:
len(unique_topics) + int(len(unique_topics) * 0.2)

50

### 2. Выделим кластеры

Проинициалазируем все необходимое для работы: (если нужно)

In [8]:
vectorizer = FasttextVectorizer()

In [9]:
kmeans_model = KMeans(n_clusters=50, random_state=42)

In [10]:
cluster_extractor = KmeansSimpleClusterExtractor(kmeans_model)

In [11]:
model = Model(vectorizer, cluster_extractor)

Подготови тексты и обучим модель:

In [12]:
texts = jokes.joke_text.values

In [13]:
texts.shape

(136338,)

In [14]:
demo = model.fit(texts)

vectorizer is fitted
clustering model is fitted


In [15]:
from sklearn.externals import joblib
joblib.dump(model.get_model(), 'knn_fasttext.pkl') 

['knn_fasttext.pkl']

In [17]:
jokes['cluster_id'] = model.get_model().labels_

In [28]:
def get_random_n_jokes(cluster_id, n):
    return jokes[jokes.cluster_id == cluster_id].sample(n)

In [29]:
unique_ids = jokes.cluster_id.unique()
df = get_random_n_jokes(unique_ids[0], 5)
for cluster_id in unique_ids[1:]:
    df = df.append(get_random_n_jokes(cluster_id, 5))
df.head()

,joke_text,cluster_id
20222,Нет смысла спорить с мужчинами — они все равно...,18
101319,Судья - пожилой свидетельнице: - Ваш возраст? ...,18
62415,"Две подруги: - Скажи тебе понравился мужчина, ...",18
54236,"- Холмс, кто же оказался в наибольшем выигрыше...",18
96366,В общественном транспорте едет женщина с детск...,18


### 3. Оценим результаты

In [31]:
df['score'] = 0
df.to_csv('scoring.csv', index=False)

In [91]:
print('\n\n'.join(df[df.cluster_id == 0].joke_text.values))

Наказание за уровень алкоголя в крови 0.05‰ (естественный уровень непьющего человека) и 5.00‰ (в стельку пьян) - одинаковые. А если нет разницы - зачем ездить трезвым?

Большинство очаровательных с первого взгляда девушек со второго взгляда оказываются разочаровательными.

Ирония - это то, что позволяет похвалить человека так, чтобы он обиделся.

Прислушивайтесь к себе, хороший человек плохого не посоветует.

в последнее время, захожу вконтакт только ради общения с одним человеком.


0 - ирония (повезло ??!?)
1 - праздники и алкоголь (немного зашумлен)
2 - спорт
3 - ШУМ
4 - пошлость, ШУМ
5 - медицина, шутки про врачей
6 - деньги
7 - ШУМ
8 - ШУМ
9 - евреи + милиция + ШУМ
10 - IT
11 - политика + менталитет + ШУМ
12 - ШУМ
13 - политика + чиновники
14 - ШУМ
15 - ШУМ
16 - алкоголь + ШУМ
17 - еда + связанное с ней
18 - сортирный юмор + отношения
19 - национальности
20 - экономика
21 - деньги + IT
22 - космос + ШУМ
23 - чиновники + коррупция
24 - купля-продажа
25 - отношения + семья
26 - ШУМ
27 - дорожное движение + ШУМ
28 - ШУМ + игра слова
29 - бизнес
30 - про анекдоты + ШУМ
31 - армия + ШУМ
32 - политика + ШУМ
33 - Россия (национальность)
34 - ШУМ
35 - ШУМ
36 - пол (м/ж)
37 - пол (м/ж) + ШУМ
38 - образование
39 - политика
40 - пол
41 - Россия + ШУМ
42 - ШУМ
43 - ирония (повезло ??!?)
44 - ШУМ
45 - за жизнь
46 - ирония + ШУМ
47 - ШУМ
48 - международные отношения
49 - пол (м/ж) + ШУМ

In [26]:
get_random_cluster_id_n_jokes(0, 5)

я настолько ревнивый и сложный человек что со мной вообще лучше не связываться.я же сожру твой мозг,чувак.

Твоя жизнь как рок-н-ролл: только без секса, наркотиков и рок-н-ролла.

- Если ли жизнь после смерти? - После такой жизни, как наша, все может быть.

Путин с Медведевым переживали-переживали за рассейский народ. Переживали-переживали... И пережили.

Часто в нашей жизни настоящее больше влияет на прошлое, чем на будущее: так пьяный милиционер, сбивший за рулем человека, оказывается уволенным из органов еще вчера.


### 4. Дальнейший анализ

найдем самый популяные слова каждого кластера

In [12]:
from sklearn.externals import joblib
model = joblib.load('knn_fasttext.pkl')

In [13]:
jokes['cluster_id'] = model.labels_

In [39]:
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
import string
analyzer = pymorphy2.MorphAnalyzer()

In [37]:
def lemmatize(text):
    text = [x for x in simple_word_tokenize(text) if x not in string.punctuation]
    text = [analyzer.parse(x)[0].normal_form for x in text]
    return ' '.join(text)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def get_top_words(cluster_id, top_n=15):
    cluster = jokes[jokes.cluster_id == cluster_id]
    cluster['lemmatized'] = cluster.joke_text.apply(lemmatize)
    tfidf = TfidfVectorizer()
    tfidf.fit(cluster.lemmatized.values)
    indices = np.argsort(tfidf.idf_)[::-1]
    features = tfidf.get_feature_names()
    top_features = [features[i] for i in indices[:top_n]]
    return top_features

In [43]:
get_top_words(10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


['ёлка',
 'очеловечиться',
 'дискуссия',
 'отрывать',
 'отрываться',
 'отсек',
 'отсидеть',
 'дискетка',
 'дискета',
 'отчётливо',
 'ох',
 'динозавр',
 'оцифровать',
 'динамик',
 'отправить']

In [44]:
import tqdm
results = []
for i in tqdm.tqdm(range(50)):
    results.append(get_top_words(i))

  0%|          | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 50/50 [16:14<00:00, 21.70s/it]


In [47]:
results[3]

['ёлы',
 'пoзнакoмиться',
 'пpемия',
 'пpиводить',
 'пpигpозить',
 'пpиглашать',
 'пpиказать',
 'пpилавок',
 'пpинести',
 'пpипоминать',
 'грань',
 'пpоyлка',
 'пpодавщица',
 'пpолетаpский',
 'пpосто']

In [49]:
with open('top-10-words-by-cluster.txt', 'w') as f:
    for i, res in enumerate(results):
        f.write('Cluster {}\n'.format(i))
        for word in res:
            f.write(word)
            f.write('\n')
        f.write('\n')